Sample Jupyter Notebook to cover 

### Identify Cryptographic Algorithms across systems and jobs

in the section 5.6 of SG248525 - Transitioning to Quantum-Safe Cryptography on IBM Z


In order to execute the Jupyter Notebook against Your own IBM CAT database there are a series of steps to perform before the Jupyter Notebook and the IBM DB2 connection will work.
***

These steps are very well described in the public IBM db2-jupyter github section.
https://github.com/IBM/db2-jupyter/tree/master

If however You just want to study the output of the sample execution then please proceed to [Query the Distribution of Keys in the Sysplex](#SampleStart)

In [ ]:
#First install the DB2 support
!/home/lyksborg/anaconda3/bin/pip install ipython-sql
!/home/lyksborg/anaconda3/bin/pip install ibm_db
!/home/lyksborg/anaconda3/bin/pip install ibm_db_sa
!/home/lyksborg/anaconda3/bin/pip install qgrid
!/home/lyksborg/anaconda3/bin/pip install itables

### You need a Db2 connect license file to connect to a z/OS database.

### Load the DB2 Extensions

In [ ]:
import ibm_db
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta 
%run db2.ipynb

### Connect to the IBM CAT Database and set the SQLID

In [ ]:
#A userID with access to the IBM CAT Database
print("Database Name: ")
dbname=input()
print("Host: ")
dbhost=input()
print("Port Number: ")
dbport=input()
print("Username: ")
dbuser=input()
#The below connect statement connects to one of our internal IBM CAT Databases
#%sql CONNECT TO DSNV133N USER $dbuser USING ? HOST winmvs3n.hursley.ibm.com PORT 43100
%sql CONNECT TO $dbname USER $dbuser USING ? HOST $dbhost PORT $dbport
print("SQL id: ")
sqlid=input()
schema = "'" + sqlid + "'"
#Schema name....
%sql SET CURRENT SQLID = $schema

### Select the sysplex to work with and the snapshot date.

In [ ]:
#Select the sysplex - systems represented in the IBM CAT Database
sysPlexSelect = "SELECT DISTINCT CAT_ENV_SYSPLEX AS SYSPLEX, CAT_ENV_SYSNAME AS SYSTEM," \
" DATE(CAT_ENV_SAMPLE_DATE) AS SNAPDATES FROM CAT_ENV"
sysPlexSystems = %sql $sysPlexSelect
print("Available Sysplexes and Systems:" )
print(sysPlexSystems.drop_duplicates())
print("Enter Sysplex ID:")
sysplex=input()
print("Enter snapshot date")
snapDate=input()
#Construct the UUID - Systemname join sub-select
SysSubSelect = "select distinct cat_env_uuid, cat_env_sysname from cat_env where cat_env_sysplex = '"\
+ sysplex + "' and date(cat_env_sample_date) = '" + snapDate + "'"

### Test the subselect..... and verify that the number uuids are as expected, eg. one per system.

In [ ]:
#Test the UUID where clause..... and verify that the number uuids are as expected, eg. one per system. 
uuidCheck = %sql select cat_env_sysname, cat_cda_uuid from cat_cda, ($SysSubSelect) where cat_cda_uuid = cat_env_uuid
print(uuidCheck.drop_duplicates())

<a id="SampleStart"></a>
# Query the distribution of keys in the sysplex

#### DES Keys

In [ ]:
#Use DB2 queries to create the data frames needed.
desKeys = %sql select cat_env_sysname, cat_des_label, cat_des_size, cat_des_kcv from cat_des, ($SysSubSelect) where cat_des_uuid = cat_env_uuid and cat_des_size <> ' '
desSize = %sql select cat_env_sysname, cat_des_size, count(cat_des_size) as sizecount from cat_des, ($SysSubSelect) where cat_des_uuid = cat_env_uuid and cat_des_size <> ' ' group by cat_env_sysname, cat_des_size
desTypeSize = %sql select cat_env_sysname, cat_des_type, cat_des_size, count(cat_des_type) as typecount from cat_des, ($SysSubSelect) where cat_des_uuid = cat_env_uuid and cat_des_size <> ' ' group by cat_env_sysname, cat_des_type, cat_des_size

In [ ]:
desKeys["CAT_ENV_SYSNAME"].value_counts()

In [ ]:
desKeysMV3A = desKeys[(desKeys['CAT_ENV_SYSNAME'].str.contains("MV3N"))]
desKeysMV3A["CAT_DES_SIZE"].value_counts().plot(kind="pie",figsize=(3,3),title="MV3N DES Keys distribution",label="DES Keys",autopct="%1.0f%%")

In [ ]:
pd.set_option('display.max_rows', 15)
desSize.pivot_table(index=['CAT_ENV_SYSNAME','CAT_DES_SIZE','SIZECOUNT'])

_DES keys count by length_
***

In [ ]:
pd.set_option('display.max_rows', 150)
desTypeSingle = desTypeSize[(desTypeSize['CAT_DES_SIZE'].str.contains("SINGLE"))]
desTypeSingle.pivot_table(index=['CAT_ENV_SYSNAME','CAT_DES_SIZE', 'CAT_DES_TYPE', 'TYPECOUNT'])

_Single length DES keys type distribution_
***

In [ ]:
pd.set_option('display.max_rows', 150)
desTypeSingle = desTypeSize[(desTypeSize['CAT_DES_SIZE'].str.contains("DOUBLE"))]
desTypeSingle.pivot_table(index=['CAT_ENV_SYSNAME','CAT_DES_SIZE', 'CAT_DES_TYPE', 'TYPECOUNT'])

_Double length DES keys type distribution_
***

In [ ]:
pd.set_option('display.max_rows', 150)
desTypeSingle = desTypeSize[(desTypeSize['CAT_DES_SIZE'].str.contains("TRIPLE"))]
desTypeSingle.pivot_table(index=['CAT_ENV_SYSNAME','CAT_DES_SIZE', 'CAT_DES_TYPE', 'TYPECOUNT'])

_Triple length DES keys type distribution_
***

In [ ]:
#Use DB2 queries to create the data frames needed.
aesKeys = %sql select cat_env_sysname, cat_sym_label, case when cat_sym_t4size = ' ' then 'unknown' else cat_sym_t4size end as cat_sym_t4size, cat_sym_kcv from cat_sym, ($SysSubSelect) where cat_sym_uuid = cat_env_uuid
aesSize = %sql select cat_env_sysname, case when cat_sym_t4size = ' ' then 'unknown' else cat_sym_t4size end as cat_sym_t4size, count(cat_sym_t4size) as sizecount from cat_sym, ($SysSubSelect) where cat_sym_uuid = cat_env_uuid group by cat_env_sysname, cat_sym_t4size
aesTypeSize = %sql select cat_env_sysname, cat_sym_type, case when cat_sym_t4size = ' ' then 'unknown' else cat_sym_t4size end as cat_sym_t4size, count(cat_sym_t4size) as typecount from cat_sym, ($SysSubSelect) where cat_sym_uuid = cat_env_uuid group by cat_env_sysname, cat_sym_type, cat_sym_t4size

In [ ]:
aesKeys["CAT_ENV_SYSNAME"].value_counts()

In [ ]:
pd.set_option('display.max_rows', 20)
aesSize.pivot_table(index=['CAT_ENV_SYSNAME','CAT_SYM_T4SIZE','SIZECOUNT'])

_AES Keys grouped by length_
***

In [ ]:
aesKeysMV3A = aesKeys[(aesKeys['CAT_ENV_SYSNAME'].str.contains("MV3N") & (aesKeys["CAT_SYM_T4SIZE"].str.contains("1|u")))]
aesKeysMV3A["CAT_SYM_T4SIZE"].value_counts().plot(kind="pie",figsize=(3,3),title="MV3N AES Keys, short or unknown length",label = "AES Keys",autopct="%1.0f%%")

_AES Keys with length not being 256 or being unknown_
***

In [ ]:
pd.set_option('display.max_rows', 150)
aesTypeShort = aesTypeSize[(aesTypeSize['CAT_SYM_T4SIZE'].str.contains("u|1"))]
aesTypeShort.pivot_table(index=['CAT_ENV_SYSNAME','CAT_SYM_T4SIZE', 'CAT_SYM_TYPE', 'TYPECOUNT'])

_AES keys type distribution_
***

# SMF
using SMF records for crypto usage

## SMF general usage statistics


Tables to look at: 
* SMF Type 82 Records: A list of ICSF SMF Records Type 82
  * `CAT_SMF31` subtype 31: Usage Statistics
  * Subtypes 40-41-42: Key Life Cycle Events
    * `CAT_SMF40` subtype 40 - CCA symmetric key lifecycle event
    * `CAT_SMF41` subtype 41 - CCA asymmetric key lifecycle event
    * `CAT_SMF42` subtype 42 - PKCS#11 object lifecycle event
  * Subtypes 44-45-46: Key Usage Events
    * `CAT_SMF44` subtype 44 - CCA symmetric key usage event
    * `CAT_SMF45` subtype 45 - CCA asymmetric key usage event
    * `CAT_SMF46` subtype 46 - PKCS#11 key usage event.
* Table of SMF type 70 subtype 2 - Cryptographic Hardware Activity
  * `CAT_SMFR7A` - Cryptographic CCA Coprocessor Data Section
  * `CAT_SMFR7B` - Cryptographic Accelerator Data Section
  * `CAT_SMFR7C` - ICSF Services Data Section
  * `CAT_SMFR7D` - Cryptographic PKCS11 Coprocessor Data Section
  
***


## Pandas dataframe with the ICSF SMF 82-31 records
Start by requesting all usage statistics from the corresponding table 

In [ ]:
#query ICSF activity for the selected plex on the selected snapshot date
smf82_31 = %sql select cat_smf31_sid, cat_smf31_uuid, cat_smf31_user_as, cat_smf31_user_tk, cat_smf31_jobid, cat_smf31_jobname, cat_smf31_reptype, cat_smf31_item, cat_smf31_count, cat_smf31_cardid, cat_smf31_stod, cat_smf31_etod from cat_smf31, ($SysSubSelect) where cat_env_uuid = cat_smf31_uuid
# remove any excessive character from the CAT_SMF31_ITEM column
smf82_31['CAT_SMF31_ITEM'] = smf82_31['CAT_SMF31_ITEM'].replace(to_replace ='[^A-Z0-9]*', value='', regex=True)
smf82_31

***
The information selected for this higlevel identifications of jobs and users that are using ICSF cryptographic services are:

- System name, CAT_SMF31_SID
- Snapshot UUID, CAT_SMF31_UUID
- User ID, CAT_SMF31_USER_AS
- Jobid, CAT_SMF31_JOBID
- Jobname, CAT_SMF31_JOBNAME
- Report type, CAT_SMF31_REPTYPE
- Item, CAT_SMF31_ITEM
- Count, CAT_SMF31_COUNT
- Card ID and type, CAT_SMF31_CARDID
- Statistic period start, CAT_SMF31_STOD
- Statistic period end, CAT_SMF31_ETOD

The report type and the item holds the information about the actual ICSF usage.
Report type ENGCARD is usage of the CEX in the system and the item in this case holds the CEX serial number and card ID holds id and type, e.g. 8C02. Count holds the number of service calls.

Report type of ENGCPACF or ENGSOFT holds information of respectivly CPACF or software cryptographic operations. In those entries the item and card id are empty. Count holds the number of service calls.
To identify jobs and users using crypto HW extract the ENGCARD report types and display these in e.g. a dataframe pivot table
***

In [ ]:
pd.set_option('display.max_rows', 10)
engusage = smf82_31[(smf82_31['CAT_SMF31_REPTYPE'].str.contains("ENGCARD")) & (smf82_31['CAT_SMF31_JOBNAME'].str.contains("ACSP") == False)]
engusage.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_CARDID','CAT_SMF31_JOBNAME'], values='CAT_SMF31_COUNT')

_Job usage of cryptographic HW_

***
And using the above as a model for extracting and displaying algorithm and service usage let us see which users and jobs are using short AES keys, DES keys, ECC keys and RSA Keys.
***

In [ ]:
pd.set_option('display.max_rows', 150)
aesusage = smf82_31[(smf82_31['CAT_SMF31_ITEM'].str.contains("AES")) & (smf82_31['CAT_SMF31_ITEM'] != 'AES256')]
aesusage.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_USER_AS','CAT_SMF31_JOBNAME'], values='CAT_SMF31_REPTYPE')

_Usage of AES keys, length 128 or 192_
***

In [ ]:
pd.set_option('display.max_rows', 15)
desusage = smf82_31[(smf82_31['CAT_SMF31_ITEM'].str.contains("DES"))]
desusage.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_USER_AS','CAT_SMF31_JOBNAME'], values='CAT_SMF31_REPTYPE')

_Usage of DES keys_
***

In [ ]:
pd.set_option('display.max_rows', 15)
eccusage = smf82_31[(smf82_31['CAT_SMF31_ITEM'].str.contains("ECC"))]
eccusage.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_USER_AS','CAT_SMF31_JOBNAME'], values='CAT_SMF31_REPTYPE')

_Usage of ECC keys_
***

In [ ]:
pd.set_option('display.max_rows', 15)
rsausage = smf82_31[(smf82_31['CAT_SMF31_ITEM'].str.contains("RSA"))]
rsausage.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_USER_AS','CAT_SMF31_JOBNAME'], values='CAT_SMF31_REPTYPE')

_Usage of RSA keys_
***

In [ ]:
pd.set_option('display.max_rows', 10)
csfalgs = smf82_31[(smf82_31['CAT_SMF31_ITEM'].str.contains("CSF")) & (smf82_31['CAT_SMF31_JOBNAME'].str.contains("CATI")) ]
csfalgs.pivot_table(index=['CAT_SMF31_SID','CAT_SMF31_ITEM','CAT_SMF31_USER_AS','CAT_SMF31_JOBNAME'], values='CAT_SMF31_REPTYPE')

_ICSF services called by CATICRA on system MV3N. CATICRA is the IBM CAT data collector job._
***

### Query Key Usage in the IBM CAT Database

In the chapter we have seen how the IBM CAT database can deliver the keys in the keystores across the sysplex.
We have displayed pivot tables to group keys by algorithm, size and type and we have displayed the distribution of key algorithms across jobs and users in the sysplex.
Some of the keys are considered to be quantum safe, e.g. AES 256 length keys.
Other keys like shorter AES keys, DES keys, RSA keys, and ECC keys are considered to be compromised by a *cryptographically relevant quantum computer (CRQC).*

Hence - we want to be able to identify usage on individual keys.

The area of digital signing is expected to be one of the first areas where a CRQC may be used to break the qryptographic security.
We will want to identyfy jobs and users that generates digital signatures and we will want to identify which keys are used to generate the signatures.

Digital signature generation in ICSF uses the service CSFDSG and privat – public keys algorithms.
So let us query usage of RSA, ECC and CRYSTALS-Dilithium usage in the SMF82-45 records.
Algorithms are in the ICSF SMF 82 records expressed in more than one way.
- In subtype 31 RSA is ‘RSA’ and in subtype 45 it is ‘07’
- In subtype 31 ECC is ‘ECC’ and in subtype 45 it is ‘09’
- In subtype 31 CRYSTALS-Dilithium is ‘LI2’ and in subtype 45 it is ‘0E’

We will now build an smf31 subtype 45 dataframe and identify users that have performed a digital signature generate.


In [ ]:
#Select all asymmetric key usage events from the corresponding table
smf82_45 = %sql SELECT CAT_ENV_SYSNAME, CAT_SMF45_STOD, CAT_SMF45_ETOD, HEX(CAT_SMF45_ALG) AS ALG, CAT_SMF45_SERVICE, CAT_SMF45_KDS_LABEL, CAT_SMF45_USECOUNT, CAT_SMF45_U_USERI FROM CAT_SMF45, ($SysSubSelect)  WHERE CAT_SMF45_UUID = CAT_ENV_UUID
smf82_45

_ICSF Service Calls for assymetric keys, SMF 82 subtype 45_
***

And below we will query only digital signature service calls

In [ ]:
dsuse = smf82_45[(smf82_45['CAT_SMF45_SERVICE'].str.contains("CSFDS"))]
dsuse.pivot_table(index=['CAT_ENV_SYSNAME','CAT_SMF45_SERVICE','CAT_SMF45_U_USERI'])

_ICSF CSFDSG and CSFDSV calls from SMF 82 subtype 45_
***

We will now look closer into the use of CSFDSG by user _Select a userID_

In [ ]:
smf82_45[(smf82_45['ALG'] == "07") & (smf82_45['CAT_SMF45_SERVICE'] == "CSFDSG  ") & (smf82_45['CAT_SMF45_U_USERI'] == "ACSPCLT1")].sort_values(by=['CAT_ENV_SYSNAME'])

_Selected userID use of CSFDSG with an RSA key, from SMF 82 subtype 45_
***



In [ ]:
smf82_45[(smf82_45['ALG'] == "09") & (smf82_45['CAT_SMF45_SERVICE'] == "CSFDSG  ") & (smf82_45['CAT_SMF45_U_USERI'] == "ACSPCLT1")].sort_values(by=['CAT_ENV_SYSNAME'])

_Selected userID use of CSFDSG with an ECC key, from SMF 82 subtype 45_
***



In [ ]:
smf82_45[(smf82_45['ALG'] == "0E") & (smf82_45['CAT_SMF45_SERVICE'] == "CSFDSG  ") & (smf82_45['CAT_SMF45_U_USERI'] == "ACSPCLT1")].sort_values(by=['CAT_ENV_SYSNAME'])

_Selected userID use of CSFDSG with a CRYSTALS-Dilithium key, from SMF 82 subtype 45_
***

And finally let us identify the coresponding SMF 82 subtype 31 records to find the job name and ID
Here we must use the recording interval of both the subtype 31 and subtype 45 records.
Either the start time or the end time of the subtype 31 record must fall within the start time and the end time of the subtype 45 record.

In [ ]:
rsasign = smf82_31[(smf82_31['CAT_SMF31_USER_TK'].str.contains("ACSPCLT1")) & (smf82_31['CAT_SMF31_ITEM'].str.contains("RSA|DSG"))]
rsadisplay = rsasign[['CAT_SMF31_SID','CAT_SMF31_USER_AS','CAT_SMF31_USER_TK','CAT_SMF31_JOBID','CAT_SMF31_STOD','CAT_SMF31_ETOD']]
rsadisplay.sort_values('CAT_SMF31_SID')

_RSA use with either CSFDSG or CSFDSV by Selected userID_
***


In [ ]:
eccsign = smf82_31[(smf82_31['CAT_SMF31_USER_TK'].str.contains("ACSPCLT1")) & (smf82_31['CAT_SMF31_ITEM'].str.contains("ECC|DSG"))]
eccdisplay = rsasign[['CAT_SMF31_SID','CAT_SMF31_USER_AS','CAT_SMF31_USER_TK','CAT_SMF31_JOBID','CAT_SMF31_STOD','CAT_SMF31_ETOD']]
eccdisplay.sort_values('CAT_SMF31_SID')

_ECC use with either CSFDSG or CSFDSV by Selected userID_
***


In [ ]:
qsasign = smf82_31[(smf82_31['CAT_SMF31_USER_TK'].str.contains("ACSPCLT1")) & (smf82_31['CAT_SMF31_ITEM'].str.contains("LI2|DSG"))]
qsadisplay = rsasign[['CAT_SMF31_SID','CAT_SMF31_USER_AS','CAT_SMF31_USER_TK','CAT_SMF31_JOBID','CAT_SMF31_STOD','CAT_SMF31_ETOD']]
qsadisplay.sort_values('CAT_SMF31_SID')

_Crystals-Dilithium use with either CSFDSG or CSFDSV by Selected userID_
***


### Close the DB2 connection

In [ ]:
%sql CONNECT CLOSE